In [ ]:
########### final code #############################

import pandas as pd
import numpy as np
import random
import customtkinter as ctk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import csv
import json
import os
# Load the dataset
try:
    df = pd.read_csv('intro_extro.csv', encoding='latin-1')  # Replace with your actual file path
except FileNotFoundError:
    print("Error: 'intro_extro.csv' not found. Please provide the correct file.")
    exit()

df_ml = df.copy()
le = LabelEncoder()
df_ml['Personality'] = le.fit_transform(df_ml['Personality']) # Encode personality labels
cols_for_ml = [col for col in df_ml.columns if col not in ['introvert_score', 'extrovert_score', 'personality']]
X = df_ml[cols_for_ml]
y = df_ml['Personality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Example hyperparameters
rf_model.fit(X_train, y_train)

corr_matrix = df.drop(columns=['Personality']).corr()
corr_matrix = df.corr()
plt.figure(figsize=(25, 20))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

def generate_questions_from_data(df, num_questions=10, synonym_replacement=True):
    if df.empty:
        return []

    cols_to_exclude = ['introvert_score', 'extrovert_score', 'Personality']
    available_cols = [col for col in df.columns if col not in cols_to_exclude]
    questions = []
    num_generated = 0

    while num_generated < num_questions and available_cols:
        col = np.random.choice(available_cols)
        available_cols.remove(col)

        words = col.split()

        if len(words) > 0 and synonym_replacement:
            synonyms = [
                {'word': 'like', 'synonyms': ['enjoy', 'favor', 'appreciate', 'prefer', 'relish']},
                {'word': 'busy', 'synonyms': ['engaged with work', 'occupied with work', 'involved in work']},
                {'word': 'parties', 'synonyms': ['gatherings', 'social events', 'get-togethers', 'socials']},
                {'word': 'easily', 'synonyms': ['readily', 'effortlessly', 'without difficulty', 'smoothly']},
                {'word': 'rules', 'synonyms': ['guidelines', 'standards', 'instructions', 'principles']},
                {'word': 'irritated', 'synonyms': ['annoyed', 'bothered', 'frustrated', 'aggravated']},
                {'word': 'visit', 'synonyms': ['explore', 'travel to', 'go to', 'tour']},
                {'word': 'new', 'synonyms': ['unfamiliar', 'novel', 'fresh', 'recent']},
                {'word': 'quiet', 'synonyms': ['reserved', 'introspective', 'undemonstrative', 'subdued']},
                {'word': 'panic', 'synonyms': ['become anxious', 'get stressed', 'feel overwhelmed', 'lose control']},
                {'word': 'order', 'synonyms': ['organization', 'structure', 'tidiness', 'arrangement']},
                {'word': 'charge', 'synonyms': ['take the lead', 'be in control', 'be proactive', 'manage']},
                {'word': 'friends', 'synonyms': ['companions', 'acquaintances', 'associates', 'buddies']},
                {'word': 'keep', 'synonyms': ['maintain', 'honor', 'fulfill', 'uphold']},
                {'word': 'feel', 'synonyms': ['sense', 'experience', 'perceive', 'detect']},
                {'word': 'trust', 'synonyms': ['believe in', 'have confidence in', 'rely on', 'depend on']},
                {'word': 'choose', 'synonyms': ['select', 'pick', 'opt for', 'decide on']},
                {'word': 'work', 'synonyms': ['labor', 'toil', 'exert oneself', 'perform tasks']},
                {'word': 'hard', 'synonyms': ['diligently', 'assiduously', 'industriously', 'earnestly']},
                {'word': 'am', 'synonyms': ['feel', 'consider yourself']}
            ]

            new_words = []
            for word in words:
                word_lower = word.lower()
                if word_lower in [item['word'] for item in synonyms]:
                    synonym_set = next((item['synonyms'] for item in synonyms if item['word'] == word_lower), None)
                    if synonym_set:
                        new_words.append(random.choice(synonym_set))
                    else:
                        new_words.append(word)
                else:
                    new_words.append(word)
            words = new_words

        question = f"{' '.join(words)}?"
        if words[0].lower() == "i":
            question = f"Do you {' '.join(words[1:])}?"
        elif words[0].lower() == "are":
            question = f"{' '.join(words)}?"
        elif words[0].lower() == "do" and len(words) > 1:
            question = f"{' '.join(words)}?"
        elif words[0].lower() == "am" and len(words) > 1:
            question = f"Are you {' '.join(words[1:])}?"
        else:
            question = f"Do you {' '.join(words)}?"

        questions.append((question, col)) # Just the question and original column name
        num_generated += 1

    return questions

def predict(df, introvert_cols, extrovert_cols):
    questions = generate_questions_from_data(df)

    if not questions:
        print("No suitable questions could be generated from the data.")
        return None

    introvert_prob = 50.0
    extrovert_prob = 50.0

    def update_probabilities(col_name, answer):
        nonlocal introvert_prob, extrovert_prob
        if answer == 'yes':
            if col_name in introvert_cols:
                introvert_prob += 5
                extrovert_prob -= 5
            elif col_name in extrovert_cols:
                introvert_prob -= 5
                extrovert_prob += 5
        elif answer == 'no':
            if col_name in introvert_cols:
                introvert_prob -= 5
                extrovert_prob += 5
            elif col_name in extrovert_cols:
                introvert_prob += 5
                extrovert_prob -= 5

    for question, col_name in questions:
        while True:
            answer = input(f"{question} (Yes/No): ").lower()
            if answer in ['yes', 'no']:
                update_probabilities(col_name, answer)
                break
            else:
                print("Invalid input. Please enter 'Yes' or 'No'.")

    introvert_prob = max(0, min(introvert_prob, 100))
    extrovert_prob = max(0, min(extrovert_prob, 100))

    print(f"Final Introvert Probability: {introvert_prob:.2f}%")
    print(f"Final Extrovert Probability: {extrovert_prob:.2f}%")

    if introvert_prob > extrovert_prob:
        print("Based on your answers, you are likely an Introvert.")
    else:
        print("Based on your answers, you are likely an Extrovert.")
    return (introvert_prob, extrovert_prob)
    
class PersonalityApp(ctk.CTk):
    def __init__(self, df):
        super().__init__()
        self.df = df
        self.title("Personality Prediction")
        self.responses = []  # List to store all responses

        # Get screen size and position the window at the center
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        window_width = 600
        window_height = 400
        x_offset = (screen_width - window_width) // 2
        y_offset = (screen_height - window_height) // 2
        self.geometry(f"{window_width}x{window_height}+{x_offset}+{y_offset}")

        self.questions = generate_questions_from_data(df)
        self.current_question_index = 0
        self.introvert_prob = 50
        self.extrovert_prob = 50
        self.result_shown = False  # Flag to track if the result is shown
        self.current_question = None  # To store the current question
        self.current_answer = None  # To store the current answer

        # Frames for question, result, and pie chart
        self.question_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.question_frame.pack(pady=100)

        self.result_frame = ctk.CTkFrame(self)
        self.chart_frame = ctk.CTkFrame(self, width=200, height=200)

        # Initially, hide the result and pie chart frames
        self.chart_frame.pack_forget()

        # Question label and Yes/No buttons
        self.question_label = ctk.CTkLabel(self.question_frame, text="", font=("Arial", 16), wraplength=500)
        self.question_label.pack(pady=(10, 30))

        self.yes_button = ctk.CTkButton(self.question_frame, text="Yes", command=lambda: self.answer_question('yes'))
        self.yes_button.pack(side="left", padx=50, pady=40)

        self.no_button = ctk.CTkButton(self.question_frame, text="No", command=lambda: self.answer_question('no'))
        self.no_button.pack(side="right", padx=50, pady=40)

        # Next button (hidden initially)
        self.next_button = ctk.CTkButton(self, text="Next", command=self.reset)
        self.next_button.pack(side="bottom", pady=20)
        self.next_button.pack_forget()  # Hide next button initially

        # Show the first question
        self.update_question()

    def reset(self):
        # Reset for next round
        self.result_shown = False  # Reset result shown flag
        self.current_question_index = 0
        self.introvert_prob = 50
        self.extrovert_prob = 50
        self.questions = generate_questions_from_data(self.df)

        # Hide the result and pie chart
        self.chart_frame.pack_forget()

        # Show the question frame and Yes/No buttons
        self.question_frame.pack(pady=70)
        self.yes_button.configure(state="normal")
        self.no_button.configure(state="normal")

        # Update question
        self.update_question()

        # Reset window size
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        window_width = 600
        window_height = 400
        x_offset = (screen_width - window_width) // 2
        y_offset = (screen_height - window_height) // 2
        self.geometry(f"{window_width}x{window_height}+{x_offset}+{y_offset}")

    def update_question(self):
        if self.current_question_index < len(self.questions):
            question, _ = self.questions[self.current_question_index]
            self.question_label.configure(text=question)
            self.current_question = question  # Store the current question
        else:
            self.show_prediction()

    def answer_question(self, answer):
        if not self.result_shown:  # Allow answering only if result is not shown yet
            if self.current_question_index < len(self.questions):
                question, col_name = self.questions[self.current_question_index]
                self.update_probabilities(col_name, answer)
                self.current_answer = answer  # Store the current answer

                # to append the json
                self.append_to_json()

            
                # Move to the next question or show results if it's the last question
                if self.current_question_index == len(self.questions) - 1:
                    self.show_prediction()
                else:
                    self.current_question_index += 1
                    self.update_question()

    def update_probabilities(self, col_name, answer):
        if answer == 'yes':
            if col_name in introvert_cols:
                self.introvert_prob += 5
                self.extrovert_prob -= 5
            elif col_name in extrovert_cols:
                self.introvert_prob -= 5
                self.extrovert_prob += 5
        elif answer == 'no':
            if col_name in introvert_cols:
                self.introvert_prob -= 5
                self.extrovert_prob += 5
            elif col_name in extrovert_cols:
                self.introvert_prob += 5
                self.extrovert_prob -= 5

    def show_prediction(self):
        # Hide question frame and Yes/No buttons
        self.question_frame.pack_forget()

        # Display result and pie chart
        result_text = f"Introvert Probability: {self.introvert_prob}%\nExtrovert Probability: {self.extrovert_prob}%"
        result_label = ctk.CTkLabel(self.result_frame, text=result_text, font=("Arial", 15))
        result_label.pack(pady=20)

        # Show pie chart
        self.show_pie_chart()

        # Show the result and pie chart frames
        self.chart_frame.pack(pady=20)

        # Increase window size to accommodate results and chart
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        window_width = 600
        window_height = 400
        x_offset = (screen_width - window_width) // 2
        y_offset = (screen_height - window_height) // 2
        self.geometry(f"{window_width}x{window_height}+{x_offset}+{y_offset}")

        # Show the Next button after answering the last question
        self.next_button.pack(side="bottom", pady=20)

        # Set the flag to indicate that the result is shown
        self.result_shown = True

        # Save all responses to a JSON file
        self.append_to_json()

    def show_pie_chart(self):
        labels = ['Introvert', 'Extrovert']
        sizes = [self.introvert_prob, self.extrovert_prob]
        colors = ['skyblue', 'lightcoral']

        fig, ax = plt.subplots(figsize=(5, 3))
        fig.patch.set_facecolor('none')
        ax.pie(sizes, labels=labels, colors=colors, autopct='%1.0f%%', shadow=True, startangle=90)
        ax.axis('equal')

        # Clear previous chart 
        for widget in self.chart_frame.winfo_children():
            widget.destroy()

        
        canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
        canvas.draw()
        canvas.get_tk_widget().pack()
        plt.close(fig)
        self.append_to_json()

    def append_to_json(self):
        file_path = "personality_results.json"
        new_data = {
            "questions": self.current_question ,
            "answer":self.current_answer,
            "final_result": {
                "introvert_prob": self.introvert_prob,
                "extrovert_prob": self.extrovert_prob
            }
        }

        try:
            with open(file_path, 'r') as f:
                existing_data = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            existing_data = []  # If file doesn't exist OR is empty/invalid JSON, start with an empty list
            
        existing_data.append(new_data)
        with open(file_path, 'w') as f:  # 'w' mode overwrites the file
            json.dump(existing_data, f, indent=4)

        # try:
        #     with open(file_path, 'r') as f:
        #         existing_data = json.load(f)
        # except (FileNotFoundError, json.JSONDecodeError):
        #     existing_data = []

        # existing_data.append(new_data)

        # with open(file_path, 'w') as f:
        #     json.dump(existing_data, f, indent=4)
        # print("Data saved to JSON file.")
        # self.after(10, self.quit)
try:
    df = pd.read_csv('intro_extro.csv', encoding='latin-1')
except FileNotFoundError:
    print("Error: 'intro_extro.csv' not found. Please provide the correct file.")
    exit()

introvert_cols = ['I am quiet around strangers.', 'Panic easily.', 'Try to follow the rules.',
                    'Get irritated easily.', 'I have little to say.']

extrovert_cols = ['Make friends easily.', 'Love large parties.', 'Like order.', 'Take charge.',
                    'Like to visit new places.']

app = PersonalityApp(df)
app.mainloop()